# Retrieval Augmented Generation(RAG) с использованием GigaChat на примере задачи "разговор с книгой"

Подход RAG позволяет большим языковым моделям (LLM) отвечать на вопросы по документам, которы не помещаются в промпт.
Ниже приведен пример того, как можно научить модель отвечать на вопросы, используя текст из книги "Мастер и Маргарита".

Подробнее про RAG вы можете прочитать в [документации LangChain](https://python.langchain.com/docs/use_cases/question_answering/) и в курсе по промпт-инженирингу от Сбера (ссылка будет позже).

В качестве примера мы рассмотрим текст романа Булгакова "Мастер и Маргарита" (главы 1 и 2).

Вопрос будет - `Какой плащ был у Понтия Пилата?`. Ответ содержится во второй главе книги:
`В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.`

## Инициализация модели
Теперь инициализируем модель GigaChat.

In [25]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [3]:
from langchain.chat_models.gigachat import GigaChat
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

Для проверки спросим у модели вопрос про цвет плаща без какого-либо контекста. Возможно, она и так будет давать ожидаемый ответ...

In [13]:
from langchain.schema import HumanMessage

question = "Какой Loss использует Yolov8?"
llm([HumanMessage(content=question)]).content[0:200]

'YOLOv8 использует несколько видов потерь для обучения модели. Однако, основной вид потери, который используется в большинстве случаев, это Cross-Entropy Loss (CEL). \n\nCross-Entropy Loss (CEL) - это фу'

Видим, что модель не отвечает так, как нам хотелось бы, поэтому применим RAG-подход.

## Подготовка документа

Для работы с документом нам нужно разделить его на части. Для этого используем `TextLoader` для загрузки книги и `RecursiveCharacterTextSplitter`, чтобы разделить текст на приблизительно равные куски в ~1000 символов с перекрытием в ~200 символов. Этот тип сплиттера сам выбирает каким способом следует оптимально разделять документ (по абзацам, по предложениям и т.д.)

In [14]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("assets/yolov8_paper.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 46


После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [15]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [16]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [17]:
print(f"... {str(docs[0])[620:800]} ...")

... tasets. We also witness YOLOv8 outperforming YOLOv5 for each RF100 category. From Figure 7b we\ncan see that YOLOv8 produces similar or even better results\ncompared to YOLOv5 [16] ...


## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [18]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [19]:
qa_chain({"query": question})

{'query': 'Какой Loss использует Yolov8?',
 'result': 'YOLOv8 использует функцию потерь, которая состоит из двух частей: локальной потери и потери классификации. Локальная потеря измеряет ошибку между предсказанными областями и истинными областями, в то время как потеря классификации измеряет ошибку между предсказанными классами и истинными классами.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [23]:
qa_chain({"query": "Архитектура Yolov8?"})

{'query': 'Архитектура Yolov8?',
 'result': 'YOLOv8 использует новую архитектуру, которая сочетает в себе модули FAN и PAN. FPN используется для генерации карт признаков на нескольких масштабах и разрешениях, в то время как PAN используется для агрегирования признаков из разных уровней сети для улучшения точности. Результаты комбинации модулей FAN и PAN лучше, чем у YOLOv5, который использует модифицированную версию архитектуры CSPDarknet, основанной на частичных связях поперек стадии (CSP), которые улучшают поток информации между различными частями сети.'}

In [24]:
qa_chain({"query": "Основные особенности Yolov8"})

{'query': 'Основные особенности Yolov8',
 'result': 'YOLOv8 имеет несколько особенностей, которые отличают его от других версий YOLO. Во-первых, он использует новую архитектуру, которая сочетает в себе модули FAN и PAN. Это позволяет ему лучше захватывать особенности на разных масштабах и разрешениях, что важно для точного обнаружения объектов разного размера и формы. Кроме того, YOLOv8 превосходит YOLOv5 по нескольким параметрам, включая более высокую метрику mAP и меньшее количество выбросов при измерении против RF100. Он также превосходит YOLOv5 для каждого RF100 категории.'}